In [1]:
from Simulation.ForwardKinematics import ForwardKinematics
import numpy as np
np.set_printoptions(precision=1, suppress=True, linewidth=200, formatter={'float': '{:8.5f}'.format})

In [2]:
fk = ForwardKinematics()

theta = np.array([1.0,1.0,1.0,1.0,1.0,1.0,1.0])
theta_dot = np.array([1,1,1,1,1,1,1])
print(fk.hessian2(theta, theta_dot))
print(fk.jacobian_dot(theta, theta_dot))

[[ 0.00000  0.00000 -0.00000 -0.00000 -0.00000  0.00000  0.00000]
 [ 0.00000 -0.54026 -0.41619  1.02365 -2.29001  1.78045 -0.82952]
 [ 0.00000 -0.84150  0.90923 -0.80474 -0.58761  1.68593 -2.73818]
 [ 0.00000  0.00000 -0.84150  0.90923 -0.80474 -0.58761  1.68593]
 [ 0.00000  0.00000  0.00000 -0.00000  0.00000 -0.00000 -0.00000]
 [ 0.00000  0.36858 -0.39824  1.50481  0.59549 -1.58675  4.49115]
 [ 0.00000 -0.23663 -0.18229  1.16491 -1.51132  3.05552  0.41254]
 [ 0.00000  0.00000  0.00000 -0.29133  0.80334 -2.11130  0.04283]]
[[ 0.00000  0.00000  0.00000  0.00000  0.00000  0.00000  0.00000]
 [ 0.00000 -0.54030 -0.41615  1.02357 -2.29004  1.78050 -0.82973]
 [ 0.00000 -0.84147  0.90930 -0.80477 -0.58746  1.68595 -2.73812]
 [ 0.00000  0.00000 -0.84147  0.90930 -0.80477 -0.58746  1.68595]
 [ 0.00000  0.00000  0.00000  0.00000 -0.00000  0.00000 -0.00000]
 [ 0.00000  0.36856 -0.39827  1.50486  0.59536 -1.58656  4.49113]
 [ 0.00000 -0.23665 -0.18227  1.16482 -1.51135  3.05554  0.41236]
 [ 0.0000

In [3]:
from neura_dual_quaternions import Quaternion

q1 = Quaternion(1,0,0,0)
q2 = Quaternion(1,0,0,0)
qm = Quaternion.slerp(q1, q2, 0)